In [15]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text, inspect, MetaData

from pathlib import Path
import pandas as pd
from itertools import product
import datetime as dt

import nxviz as nv
import networkx as nx
from nxviz import layouts, plots, lines
from nxviz import nodes, edges, annotate, highlights
from nxviz.plots import despine, rescale, respine, aspect_equal

from nxviz.utils import edge_table, node_table
from nxviz import encodings as aes

In [37]:
basefolder = Path.home().joinpath("Documents", "data", "opendatasus")
dbname = "SIM_WAREHOUSE.db"
dbname = "SIH_CNES_WAREHOUSE.db"

In [38]:
engine = create_engine(f'sqlite:///'+str(basefolder.joinpath(dbname)))

In [47]:
# -- test

def query_metadata(engine):
    inspector = inspect(engine)
    tables = inspector.get_table_names()
    table_dict = { table_name : inspector.get_columns(table_name) for table_name in tables }
    return table_dict


def query_data(query_str, table_name, engine, batchsize=1000):

    schema_data = {
        'rows': [],
        'columns': [],
    }

    with engine.connect() as conn:
        qres = conn.execute(query_str)
        schema_data['columns'] = list(qres.keys())

        while True:
            rows = qres.fetchmany(batchsize)
            if not rows:
                break
            schema_data["rows"] += [ row for row in rows ]
    
    res_df = pd.DataFrame(schema_data['rows'], columns=schema_data['columns'])
    return res_df

In [48]:
# teste -- SIM --
#db_metadata = query_metadata(engine)
#db_metadata

In [64]:
query_str = f'''
    SELECT * FROM sim LIMIT 5
'''

df = query_data(query_str, 'sim', engine)
print(df.columns)
print(df.info())
df.head(3)

AttributeError: 'str' object has no attribute 'connect'

In [56]:
query_causa = f'''
    SELECT 
        CHAVE_CONTADOR_FONTE, CODMUNRES, DTOBITO, 
        SUBSTR(CAUSABAS, 1, 3) AS CAUSABAS, LINHAA, LINHAB, LINHAC, 
        LINHAD, LINHAII
    FROM
        sim
    WHERE DTOBITO >= '2015-01-01' AND DTOBITO <= '2015-12-31'
'''

df = query_data(query_causa, 'sim', engine)
print(df.columns)
print(df.info())
df.head(3)

Index(['CHAVE_CONTADOR_FONTE', 'CODMUNRES', 'DTOBITO', 'CAUSABAS', 'LINHAA',
       'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260715 entries, 0 to 1260714
Data columns (total 9 columns):
 #   Column                Non-Null Count    Dtype 
---  ------                --------------    ----- 
 0   CHAVE_CONTADOR_FONTE  1260715 non-null  object
 1   CODMUNRES             1260715 non-null  object
 2   DTOBITO               1260715 non-null  object
 3   CAUSABAS              1260715 non-null  object
 4   LINHAA                1219299 non-null  object
 5   LINHAB                979520 non-null   object
 6   LINHAC                618600 non-null   object
 7   LINHAD                267688 non-null   object
 8   LINHAII               423081 non-null   object
dtypes: object(9)
memory usage: 86.6+ MB
None


,CHAVE_CONTADOR_FONTE,CODMUNRES,DTOBITO,CAUSABAS,LINHAA,LINHAB,LINHAC,LINHAD,LINHAII
0,1DOAC2015,120060,2015-11-21 00:00:00.000000,R99,*R99X,None,None,None,None
1,2DOAC2015,120050,2015-02-02 00:00:00.000000,E14,*I10X,*E149,None,None,None
2,3DOAC2015,120050,2015-02-08 00:00:00.000000,J43,*J439,None,None,None,None


In [65]:
def define_edgelist(sim_df):
    '''

    '''
    cols = ["CAUSABAS", "LINHAA", "LINHAB", "LINHAC", "LINHAD", "LINHAII"]
    subset_df = df[cols].copy()

    subset_lst = subset_df.apply(lambda x: list(x), axis=1).tolist()
    aux = []
    for current_list in subset_lst:
        new_el = []
        for element in current_list:
            if pd.notna(element):
                cur = [ el for el in element.strip().split("*") if el!='' ]
                new_el += cur
        aux.append([ el[:3] for el in  new_el ] )

    # -- create list of pairs (order not relevant)
    list_of_pairs = []
    for curr in aux:
        pairs = list(set(tuple(sorted(t)) for t in product(curr, curr) if t[0] != t[1]))
        list_of_pairs += pairs

    edgelist = pd.Series(list_of_pairs).value_counts().reset_index()
    edgelist['source'] = edgelist['index'].apply(lambda x: x[0])
    edgelist['target'] = edgelist['index'].apply(lambda x: x[1])
    edgelist = edgelist.drop('index', axis=1)
    return edgelist

def create_network(edgelist, cutoff=0):
    edgelist1 = edgelist[edgelist['count']>cutoff]

    nodenames = edgelist1['source'].unique().tolist() + edgelist1['target'].unique().tolist()
    nodenames = list(set(nodenames))
    nodelabels = { nodenames[label]: label for label in range(len(nodenames)) }
    
    node_metadata = []
    for label, node in enumerate(nodenames):
        node_metadata.append((
            label, { 'cid10': nodenames[label] }
        ))
    
    edge_metadata = []
    for edge, row in edgelist1.iterrows():
        edge_metadata.append(
            (nodelabels[row['source']], nodelabels[row['target']], {'count': row['count']})
        )
    
    graph = nx.Graph()
    
    graph.add_nodes_from(node_metadata)
    graph.add_edges_from(edge_metadata)
    return graph


In [62]:
edgelist = define_edgelist(df)

In [63]:
edgelist

,count,source,target
0,105335,A41,J18
1,55083,E14,I10
2,51303,J18,J96
3,42875,I10,I21
4,35812,A41,I10
...,...,...,...
114341,1,B23,E72
114342,1,B23,R10
114343,1,R09,W22
114344,1,J06,S06


In [66]:
graph = create_network(edgelist, cutoff=5)

In [68]:
print(graph.number_of_nodes(), graph.number_of_edges())

1235 31517


## **SIH Networks**

In [68]:
basefolder = Path.home().joinpath("Documents", "data", "opendatasus")
dbname = "SIH_CNES_WAREHOUSE.db"

In [69]:
engine = create_engine(f'sqlite:///'+str(basefolder.joinpath(dbname)))

In [70]:
# -- test

def query_metadata(engine):
    inspector = inspect(engine)
    tables = inspector.get_table_names()
    table_dict = { table_name : inspector.get_columns(table_name) for table_name in tables }
    return table_dict


def query_data(query_str, engine, batchsize=1000):

    schema_data = {
        'rows': [],
        'columns': [],
    }

    with engine.connect() as conn:
        qres = conn.execute(query_str)
        schema_data['columns'] = list(qres.keys())

        while True:
            rows = qres.fetchmany(batchsize)
            if not rows:
                break
            schema_data["rows"] += [ row for row in rows ]
    
    res_df = pd.DataFrame(schema_data['rows'], columns=schema_data['columns'])
    return res_df

In [89]:
def select_period_aih(engine, start_date, final_date, diag_level=0):
    '''
        ...
    '''
    if diag_level>4:
        diag_level = 4
    elif diag_level<0:
        diag_level = 0
    
    query = f'''
        SELECT 
            N_AIH, CNES, MUNIC_RES, MUNIC_MOV,
            SUBSTR(DIAG_PRINC,1,{diag_level}) as DIAG_CATEG 
        FROM aih_reduzida
        WHERE DT_INTER >= '{start_date.strftime("%Y-%m-%d")}' AND DT_INTER <= '{final_date.strftime("%Y-%m-%d")}'
    '''
    df = query_data(query, engine)
    return df

def select_period_servicos(engine, start_date, final_date, proc_level=6):
    '''
        ...
    '''
    query = f'''
        SELECT
            *
        FROM (
            SELECT 
                a.SP_NAIH, a.SP_CNES, a.SP_ATOPROF, 
                b.MUNIC_RES, b.MUNIC_MOV, b.DT_INTER
            FROM servicos_profissionais a
            LEFT JOIN aih_reduzida b
            ON a.SP_NAIH = b.N_AIH
        )
        WHERE DT_INTER >= '{start_date.strftime("%Y-%m-%d")}' AND DT_INTER <= '{final_date.strftime("%Y-%m-%d")}'
    '''
    df = query_data(query, engine)
    return df


def edgelist_for_citytocity(engine, start_date, final_date, diag_level):
    '''
        ...
    '''
    df = select_period_aih(engine, start_date, final_date, diag_level=diag_level)
    edgelist = df.groupby(["MUNIC_RES", "MUNIC_MOV"])["DIAG_CATEG"].value_counts().reset_index()
    edgelist = pd.pivot_table(edgelist, index=["MUNIC_RES", "MUNIC_MOV"], columns="DIAG_CATEG", values="count").fillna(0)
    edgelist["SOMA"] = edgelist.apply(sum, axis=1)
    return edgelist

def edgelist_for_citytohospital(engine, start_date, final_date, diag_level):
    '''
        ...
    '''
    df = select_period_aih(engine, start_date, final_date, diag_level=diag_level)
    edgelist = df.groupby(["MUNIC_RES", "CNES"])["DIAG_CATEG"].value_counts().reset_index()
    edgelist = pd.pivot_table(edgelist, index=["MUNIC_RES", "CNES"], columns="DIAG_CATEG", values="count").fillna(0)
    edgelist["SOMA"] = edgelist.apply(sum, axis=1)
    return edgelist

In [82]:
start_date, final_date = dt.datetime(2010,1,1), dt.datetime(2010,2,1)

df = select_period_aih(engine, start_date, final_date, diag_level=1)

In [83]:
df

,N_AIH,CNES,MUNIC_RES,MUNIC_MOV,DIAG_CATEG
0,2310100904265,2333899,230470,230470,O
1,2310100904276,2333899,230470,230470,O
2,2310100904287,2333899,230470,230470,O
3,2310100904298,2333899,230470,230470,O
4,2310100904309,2333899,230470,230470,O
...,...,...,...,...,...
40265,2310104444450,2526638,230440,230440,J
40266,2310104450511,2499363,230440,230440,O
40267,2310104538192,2526638,230440,230440,J
40268,2310104728646,2478161,230440,230440,S


In [90]:
start_date, final_date = dt.datetime(2010,1,1), dt.datetime(2010,2,1)

df = select_period_servicos(engine, start_date, final_date)

In [91]:
df

,SP_NAIH,SP_CNES,SP_ATOPROF,MUNIC_RES,MUNIC_MOV,DT_INTER
0,2310100905827,2552345,0305020021,230180,230180,2010-01-13 00:00:00.000000
1,2310100904012,2333899,0310010039,230470,230470,2010-01-22 00:00:00.000000
2,2310100904012,2333899,0301010145,230470,230470,2010-01-22 00:00:00.000000
3,2310100904012,2333899,0202031179,230470,230470,2010-01-22 00:00:00.000000
4,2310100904012,2333899,0310010039,230470,230470,2010-01-22 00:00:00.000000
...,...,...,...,...,...,...
300436,2310100083929,2480026,0303170077,230440,230440,2010-01-05 00:00:00.000000
300437,2310100083929,2480026,0303170077,230440,230440,2010-01-05 00:00:00.000000
300438,2310100083929,2480026,0303170077,230440,230440,2010-01-05 00:00:00.000000
300439,2310100065944,2529459,0303170093,231025,230440,2010-01-14 00:00:00.000000


In [94]:
edgelist = df.groupby(["MUNIC_RES", "MUNIC_MOV"])["SP_ATOPROF"].value_counts().reset_index()
edgelist = pd.pivot_table(edgelist, index=["MUNIC_RES", "MUNIC_MOV"], columns="SP_ATOPROF", values="count").fillna(0)

In [95]:
edgelist

,MUNIC_RES,MUNIC_MOV,SP_ATOPROF,count
0,120040,230440,0303010010,2
1,120040,230440,0802010040,1
2,120040,230440,0301010170,1
3,120040,230440,0202010643,1
4,120040,230440,0202010635,1
...,...,...,...,...
35087,510340,230440,0203020030,1
35088,510340,230440,0205020038,1
35089,510340,230440,0302040021,1
35090,510340,230440,0202010201,1


In [84]:
edgelist = edgelist_for_citytocity(engine, start_date, final_date, diag_level=0)
edgelist

DIAG_CATEG              SOMA
MUNIC_RES MUNIC_MOV         
120040    230440     1     1
130260    230190     1     1
          230440     5     5
          231340     1     1
140010    230440     1     1
...                 ..   ...
355030    230730     1     1
410690    231330     1     1
430210    230440     2     2
510340    230440     1     1
          231330     1     1

[824 rows x 2 columns]

In [86]:
'' in edgelist.columns

True

In [74]:
edgelist.sort_values(by="SOMA", ascending=False)

DIAG_CATEG               A      B      C      D      E      F     G      H  \
MUNIC_RES MUNIC_MOV                                                          
230440    230440     675.0  260.0  515.0  326.0  148.0  495.0  99.0  200.0   
231290    231290      67.0   53.0   31.0   32.0   28.0   70.0   6.0    2.0   
230730    230730     144.0   11.0    8.0   23.0   22.0    0.0   4.0    1.0   
230420    230420      82.0   19.0    3.0   29.0   24.0   79.0   5.0    0.0   
230370    230370      16.0    5.0    6.0   20.0   25.0    0.0  12.0    0.0   
...                    ...    ...    ...    ...    ...    ...   ...    ...   
230840    230320       0.0    1.0    0.0    0.0    0.0    0.0   0.0    0.0   
          230440       0.0    0.0    0.0    0.0    0.0    0.0   0.0    0.0   
230850    230360       0.0    0.0    0.0    0.0    0.0    0.0   0.0    0.0   
          230420       0.0    0.0    0.0    0.0    0.0    0.0   0.0    0.0   
510340    231330       0.0    0.0    0.0    0.0    0.0    0.0   0.0    0.0   

DIAG_CATEG               I       J  ...      P     Q      R      S      T  \
MUNIC_RES MUNIC_MOV                 ...                                     
230440    230440     932.0  1182.0  ...  328.0  94.0  126.0  753.0  219.0   
231290    231290      85.0   109.0  ...   13.0   4.0   22.0   50.0   12.0   
230730    230730      41.0   144.0  ...   30.0   5.0   36.0   40.0   35.0   
230420    230420     108.0    83.0  ...   11.0   6.0    4.0   47.0   13.0   
230370    230370      12.0    63.0  ...   14.0   0.0    2.0   50.0   13.0   
...                    ...     ...  ...    ...   ...    ...    ...    ...   
230840    230320       0.0     0.0  ...    0.0   0.0    0.0    0.0    0.0   
          230440       0.0     0.0  ...    0.0   0.0    0.0    0.0    1.0   
230850    230360       0.0     0.0  ...    0.0   0.0    0.0    0.0    0.0   
          230420       0.0     0.0  ...    0.0   0.0    0.0    1.0    0.0   
510340    231330       0.0     0.0  ...    0.0   0.0    0.0    0.0    0.0   

DIAG_CATEG             W    X    Y      Z     SOMA  
MUNIC_RES MUNIC_MOV                                 
230440    230440     0.0  0.0  4.0  397.0  11330.0  
231290    231290     0.0  0.0  0.0   31.0   1093.0  
230730    230730     0.0  0.0  0.0    6.0   1048.0  
230420    230420     0.0  0.0  0.0    5.0    865.0  
230370    230370     0.0  0.0  0.0   41.0    703.0  
...                  ...  ...  ...    ...      ...  
230840    230320     0.0  0.0  0.0    0.0      1.0  
          230440     0.0  0.0  0.0    0.0      1.0  
230850    230360     0.0  0.0  0.0    0.0      1.0  
          230420     0.0  0.0  0.0    0.0      1.0  
510340    231330     0.0  0.0  0.0    0.0      1.0  

[824 rows x 25 columns]

In [75]:
edges = [(src, tgt, attr.to_dict()) for (src, tgt), attr in edgelist.iterrows()]

In [77]:
g = nx.DiGraph()
g.add_edges_from(edges)

In [79]:
for edge in g.edges(data=True):
    print(edge)

('120040', '230440', {'A': 1.0, 'B': 0.0, 'C': 0.0, 'D': 0.0, 'E': 0.0, 'F': 0.0, 'G': 0.0, 'H': 0.0, 'I': 0.0, 'J': 0.0, 'K': 0.0, 'L': 0.0, 'M': 0.0, 'N': 0.0, 'O': 0.0, 'P': 0.0, 'Q': 0.0, 'R': 0.0, 'S': 0.0, 'T': 0.0, 'W': 0.0, 'X': 0.0, 'Y': 0.0, 'Z': 0.0, 'SOMA': 1.0})
('230440', '230350', {'A': 0.0, 'B': 0.0, 'C': 0.0, 'D': 0.0, 'E': 0.0, 'F': 0.0, 'G': 0.0, 'H': 0.0, 'I': 0.0, 'J': 0.0, 'K': 0.0, 'L': 0.0, 'M': 0.0, 'N': 0.0, 'O': 1.0, 'P': 0.0, 'Q': 0.0, 'R': 0.0, 'S': 0.0, 'T': 0.0, 'W': 0.0, 'X': 0.0, 'Y': 0.0, 'Z': 0.0, 'SOMA': 1.0})
('230440', '230370', {'A': 1.0, 'B': 0.0, 'C': 0.0, 'D': 0.0, 'E': 0.0, 'F': 0.0, 'G': 0.0, 'H': 0.0, 'I': 0.0, 'J': 0.0, 'K': 1.0, 'L': 0.0, 'M': 0.0, 'N': 0.0, 'O': 12.0, 'P': 0.0, 'Q': 0.0, 'R': 0.0, 'S': 4.0, 'T': 2.0, 'W': 0.0, 'X': 0.0, 'Y': 0.0, 'Z': 0.0, 'SOMA': 20.0})
('230440', '230428', {'A': 0.0, 'B': 0.0, 'C': 0.0, 'D': 0.0, 'E': 0.0, 'F': 0.0, 'G': 0.0, 'H': 0.0, 'I': 0.0, 'J': 0.0, 'K': 0.0, 'L': 0.0, 'M': 0.0, 'N': 1.0, 'O': 4.0

In [81]:
print(g.nodes())

['120040', '230440', '130260', '230190', '231340', '140010', '140047', '150080', '150140', '230940', '172100', '230360', '210140', '210637', '210860', '211130', '220027', '230420', '220202', '230410', '220209', '220545', '230523', '220720', '221040', '221093', '221100', '231290', '221120', '230010', '230250', '230840', '230015', '230765', '230770', '231160', '230020', '230425', '230030', '230550', '231135', '230040', '231330', '230050', '230060', '230070', '230760', '230075', '230640', '230080', '230090', '230370', '231070', '230100', '230350', '230428', '230625', '230110', '230535', '231180', '230120', '230210', '230125', '230130', '230730', '230140', '230150', '230160', '230170', '230180', '230540', '230570', '231400', '230185', '231130', '230195', '230200', '230830', '230205', '230260', '230220', '230230', '230780', '230240', '230280', '230270', '230740', '230290', '230300', '230310', '230900', '230320', '230330', '230340', '231230', '230365', '231220', '230380', '230390', '230393',